# GHGEmissions Training

## Imports

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import torch
import random
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as album
from PIL import Image
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from sklearn.metrics import accuracy_score
import plotly.express as px
import torchmetrics
from torchmetrics import MeanAbsolutePercentageError


%matplotlib inline

## Global Variables

In [ ]:
GHG_EMISSIONS_PATH = '/home/ah2719/FYP/Spatial_Finance_Transport/data/ghg_emissions.csv'

NN_MODEL_PATH = "/home/ah2719/FYP/Spatial_Finance_Transport/models/nn_model.pth"

## General Functions

## Model

In [ ]:
class CustomDataset(Dataset):
    def __init__(self):
        self.labels = torch.tensor(pd.read_csv(GHG_EMISSIONS_PATH)['Annual Territorial emissions (kt CO2e)'].values.astype('float32'))
        self.population = torch.tensor(pd.read_csv(GHG_EMISSIONS_PATH)['Mid-year Population (thousands)'].values.astype('float32')).unsqueeze(1) # Training
        #self.speed_data = pd.read_csv(SPEED_DATA_PATH) 
        #self.road_width = pd.read_csv(ROAD_WIDTH_PATH)
        self.area = torch.tensor(pd.read_csv(GHG_EMISSIONS_PATH)['Area (km2)'].values.astype('float32')).unsqueeze(1)
        self.motorways = torch.tensor(pd.read_csv(GHG_EMISSIONS_PATH)['Motorways'].values.astype('float32')).unsqueeze(1)
        self.a_roads = torch.tensor(pd.read_csv(GHG_EMISSIONS_PATH)['A roads'].values.astype('float32')).unsqueeze(1)
        self.minor_roads = torch.tensor(pd.read_csv(GHG_EMISSIONS_PATH)['Minor Roads'].values.astype('float32')).unsqueeze(1)

        self.x = torch.concat((self.population, self.area, self.motorways, self.a_roads, self.minor_roads), dim=-1)
        self.y = self.labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

custom_data = CustomDataset()
train_split = 0.8
train_data, test_data = random_split(custom_data, [train_split, 1-train_split])